In [4]:
from dataWrangler import Process,Spell,Replace,Outliers
from scrape_craigslist import Scrape
from predictions import singlePredict
import urllib.request
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import requests
import datetime
import glob   
import os
import io
import re

In [2]:
li=[]
for filename in glob.glob(r"..\UsedCar-PriceEstimator\data\*.csv"):
    collection = pd.read_csv(filename, index_col=None, header=0)
    li.append(collection)
frame = pd.concat(li, axis=0, ignore_index=True)

In [13]:
singlePredict(frame,'https://sfbay.craigslist.org/sby/cto/d/san-jose-smog-02-toyota-sequoia-sr5/7269144023.html')

toyota sequoia Accuracy: 97.72 %.
predicted: 5494 actual: 6000


In [ ]:
#import pandas_profiling
#profile=df.profile_report()